In [4]:
import pandas as pd
import numpy as np
import ta 
import os

# Loading merged dataset of closing prices
df = pd.read_csv(rf"C:\Users\ansul\OneDrive\Desktop\data science project\portfolio_optimization_ML\data\raw\merged_close.csv", parse_dates=["Date"])
df.set_index("Date", inplace=True)

df.head()

,AAPL,MSFT,AMZN,TSLA,JPM
Date,,,,,
2018-01-02,40.381001,79.198357,59.450500,21.368668,87.152008
2018-01-03,40.373959,79.566910,60.209999,21.150000,87.240829
2018-01-04,40.561501,80.267212,60.479500,20.974667,88.490601
2018-01-05,41.023308,81.262383,61.457001,21.105333,87.922516
2018-01-08,40.870926,81.345299,62.343498,22.427334,88.052368


In [7]:
## making a directory for processed data
os.makedirs(r"C:\Users\ansul\OneDrive\Desktop\data science project\portfolio_optimization_ML\data\processed", exist_ok=True)

In [8]:
# Computing daily returns
returns = df.pct_change().dropna()
returns.to_csv(rf"C:\Users\ansul\OneDrive\Desktop\data science project\portfolio_optimization_ML\data\processed\daily_returns.csv")

In [10]:
# Feature engineering (per ticker)
features = {}

for ticker in df.columns:
    temp = pd.DataFrame(index=df.index)
    temp["Close"] = df[ticker]
    
    # Returns
    temp["Return"] = returns[ticker]
    
    # Rolling mean & volatility
    temp["SMA_10"] = temp["Close"].rolling(10).mean()
    temp["SMA_20"] = temp["Close"].rolling(20).mean()
    temp["Volatility_10"] = temp["Return"].rolling(10).std()
    
    # RSI (Relative Strength Index)
    temp["RSI"] = ta.momentum.RSIIndicator(temp["Close"], window=14).rsi()
    
    # MACD (trend indicator)
    macd = ta.trend.MACD(temp["Close"])
    temp["MACD"] = macd.macd()
    temp["MACD_Signal"] = macd.macd_signal()
    
    # Drop NaNs
    temp = temp.dropna()
    
    features[ticker] = temp

    # Save per ticker
    temp.to_csv(rf"C:\Users\ansul\OneDrive\Desktop\data science project\portfolio_optimization_ML\data\processed\{ticker}_features.csv")

    print(f"Saved features for {ticker} with shape {temp.shape}")

#  Example preview
print("\nSample features for AAPL:")
print(pd.read_csv(rf"C:\Users\ansul\OneDrive\Desktop\data science project\portfolio_optimization_ML\data\processed\AAPL_features.csv").head())

Saved features for AAPL with shape (1895, 8)
Saved features for MSFT with shape (1895, 8)
Saved features for AMZN with shape (1895, 8)
Saved features for TSLA with shape (1895, 8)
Saved features for JPM with shape (1895, 8)

Sample features for AAPL:
         Date      Close    Return     SMA_10     SMA_20  Volatility_10  \
0  2018-02-20  40.449123 -0.003364  38.692767  39.049406       0.024284   
1  2018-02-21  40.265541 -0.004539  38.897591  38.987608       0.021860   
2  2018-02-22  40.602127  0.008359  39.217886  38.975691       0.019719   
3  2018-02-23  41.308243  0.017391  39.711702  39.035533       0.015226   
4  2018-02-26  42.125000  0.019772  40.242707  39.131524       0.015374   

         RSI      MACD  MACD_Signal  
0  56.689537 -0.148416    -0.562798  
1  55.311633 -0.060069    -0.462252  
2  57.358090  0.036684    -0.362465  
3  61.356182  0.168398    -0.256293  
4  65.397520  0.334828    -0.138069  
